## I love to debug

In [187]:
import functools
import os
import time
from typing import List
import matplotlib.pyplot as plt
import cirq
import networkx as nx
import numpy as np
import qiskit
from mitiq import benchmarks, pec

from qiskit_ibm_runtime.fake_provider import FakeTorino, FakeSherbrooke

from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)

from qiskit import QuantumCircuit


In [188]:
backend= FakeSherbrooke()
qc = QuantumCircuit(2)
qc.cx(1,0)

pm = qiskit.transpiler.generate_preset_pass_manager(optimization_level=0, backend=backend)
transpiled_circuit = pm.run(qc)
transpiled_circuit.draw()




global phase: π/2
         ┌─────────┐ ┌─────────┐┌────┐┌─────────┐┌─────────┐┌──────┐     
q_0 -> 0 ┤ Rz(π/2) ├─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ Rz(π/2) ├┤1     ├─────
         ├─────────┴┐└─────────┘└────┘└─────────┘└─────────┘│  Ecr │┌───┐
q_1 -> 1 ┤ Rz(-π/2) ├───────────────────────────────────────┤0     ├┤ X ├
         └──────────┘                                       └──────┘└───┘

In [189]:
!pip list -- local

Package                           Version
--------------------------------- ------------------
aext-assistant                    4.1.0
aext-assistant-server             4.1.0
aext-core                         4.1.0
aext-core-server                  4.1.0
aext-panels                       4.1.0
aext-panels-server                4.1.0
aext-project-filebrowser-server   4.1.0
aext-share-notebook               4.1.0
aext-share-notebook-server        4.1.0
aext-shared                       4.1.0
aext-toolbox                      4.1.0
aiobotocore                       2.12.3
aiohappyeyeballs                  2.4.0
aiohttp                           3.10.5
aioitertools                      0.7.1
aiosignal                         1.2.0
alabaster                         0.7.16
alembic                           1.13.3
altair                            5.0.1
anaconda-anon-usage               0.4.4
anaconda-auth                     0.8.4
anaconda-catalogs                 0.2.0
anaconda-cli-base    

In [190]:
print(FakeTorino.num_qubits)
print(FakeSherbrooke.num_qubits)

In [191]:
ideal_backend = qiskit_aer.AerSimulator()
noisy_backend = FakeSherbrooke()
# Random seed for circuit generation.
seed: int = 1

# Display verbose output.
verbose: bool = False
# Give queue updates every this many seconds when running on hardware device.
verbose_update_time: int = 30

In [192]:
def get_phys_qubits(n_qubits):
    ibm_sherbrooke_ordering = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 17, 30, 29, 28, 27, 26, 25, 24, 
                            23, 22, 21, 20, 33, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 55, 68, 
                            67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 71, 77, 78, 79, 80, 81, 82, 83, 
                            84, 85, 86, 87, 93, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 
                            109, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126] # Up to 87 qubits
    
    return ibm_sherbrooke_ordering[: n_qubits]

In [193]:
def get_layout(n_qubits):
    phys_qubits = get_phys_qubits(n_qubits)
    virt_qubits =[]
    for qubit_i in range(0,n_qubits):
        virt_qubits.append(qubit_i)
    layout =  dict(zip(phys_qubits, virt_qubits))
        
    return layout
        
    

In [220]:
print(get_layout(15))

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 17: 13, 30: 14}


In [ ]:
def get_computer(n_qubits):
    layout = get_layout(n_qubits)
    phys_edges = noisy_backend.coupling_map.get_edges()
    virt_edges = []
    for edge_i in range(0, int(len(phys_edges)/2)):
        phys_edge = phys_edges[edge_i]
        print(phys_edge)
        try:
            virt_edge = (layout[phys_edge[0]],layout[phys_edge[1]])
        except:
            print("This edge is not included in our layout")
        virt_edges.append(virt_edge)
        
    # Make connectivity graph 
    computer = nx.Graph()
    computer.add_edges_from(virt_edges[:n_qubits - 1])

    # # Add reversed edges to computer graph.
    # # This is important to represent CNOT gates with target and control reversed.
    # computer = nx.to_directed(computer)
    return computer

SyntaxError: invalid syntax (242495389.py, line 3)

In [224]:
print(get_computer(6))
print(noisy_backend.coupling_map.get_edges()[:6])
print(len(noisy_backend.coupling_map.get_edges()[:6]))
print(noisy_backend.coupling_map.get_edges()[0][0])
print(GenericBackendV2(20).coupling_map.get_edges())

Graph with 6 nodes and 5 edges
[(1, 0), (1, 2), (3, 2), (4, 3), (4, 15), (5, 4)]
6
1
EdgeList[(0, 1), (1, 0), (0, 2), (2, 0), (0, 3), (3, 0), (0, 4), (4, 0), (0, 5), (5, 0), (0, 6), (6, 0), (0, 7), (7, 0), (0, 8), (8, 0), (0, 9), (9, 0), (0, 10), (10, 0), (0, 11), (11, 0), (0, 12), (12, 0), (0, 13), (13, 0), (0, 14), (14, 0), (0, 15), (15, 0), (0, 16), (16, 0), (0, 17), (17, 0), (0, 18), (18, 0), (0, 19), (19, 0), (1, 2), (2, 1), (1, 3), (3, 1), (1, 4), (4, 1), (1, 5), (5, 1), (1, 6), (6, 1), (1, 7), (7, 1), (1, 8), (8, 1), (1, 9), (9, 1), (1, 10), (10, 1), (1, 11), (11, 1), (1, 12), (12, 1), (1, 13), (13, 1), (1, 14), (14, 1), (1, 15), (15, 1), (1, 16), (16, 1), (1, 17), (17, 1), (1, 18), (18, 1), (1, 19), (19, 1), (2, 3), (3, 2), (2, 4), (4, 2), (2, 5), (5, 2), (2, 6), (6, 2), (2, 7), (7, 2), (2, 8), (8, 2), (2, 9), (9, 2), (2, 10), (10, 2), (2, 11), (11, 2), (2, 12), (12, 2), (2, 13), (13, 2), (2, 14), (14, 2), (2, 15), (15, 2), (2, 16), (16, 2), (2, 17), (17, 2), (2, 18), (18, 2), 

In [244]:
n_qubits = 136
layout = get_layout(n_qubits)
print(layout)
print(layout[17])
phys_edges = noisy_backend.coupling_map.get_edges()
print(phys_edges)
print(phys_edges[1])

virt_edges = []
for edge_i in range(0, int(len(phys_edges)/2)):
    phys_edge = phys_edges[edge_i]
    print(phys_edge)
    try:
        virt_edge = (layout[phys_edge[0]],layout[phys_edge[1]])
    except:
        print("This edge is not included in our layout")
    print(phys_edge)
    virt_edges.append(virt_edge)

print(virt_edges)
# Make connectivity graph 
computer = nx.Graph()
computer.add_edges_from(virt_edges[:n_qubits - 1])

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 17: 13, 30: 14, 29: 15, 28: 16, 27: 17, 26: 18, 25: 19, 24: 20, 23: 21, 22: 22, 21: 23, 20: 24, 33: 25, 39: 26, 40: 27, 41: 28, 42: 29, 43: 30, 44: 31, 45: 32, 46: 33, 47: 34, 48: 35, 49: 36, 55: 37, 68: 38, 67: 39, 66: 40, 65: 41, 64: 42, 63: 43, 62: 44, 61: 45, 60: 46, 59: 47, 58: 48, 71: 49, 77: 50, 78: 51, 79: 52, 80: 53, 81: 54, 82: 55, 83: 56, 84: 57, 85: 58, 86: 59, 87: 60, 93: 61, 106: 62, 105: 63, 104: 64, 103: 65, 102: 66, 101: 67, 100: 68, 99: 69, 98: 70, 97: 71, 96: 72, 109: 73, 114: 74, 115: 75, 116: 76, 117: 77, 118: 78, 119: 79, 120: 80, 121: 81, 122: 82, 123: 83, 124: 84, 125: 85, 126: 86}
13
EdgeList[(1, 0), (1, 2), (3, 2), (4, 3), (4, 15), (5, 4), (6, 5), (7, 6), (7, 8), (8, 9), (10, 9), (10, 11), (11, 12), (12, 13), (14, 0), (14, 18), (16, 8), (17, 12), (17, 30), (18, 19), (19, 20), (20, 33), (21, 20), (21, 22), (22, 15), (23, 22), (23, 24), (25, 24), (26, 16), (26, 25), (26, 27), (2

In [197]:
n_qubits=5
depth = 3
trials = 3
shots = 10000
num_samples = 100

computer= get_computer(n_qubits)
print(computer)

noisy_backend = FakeSherbrooke()
ideal_backend = qiskit_aer.AerSimulator()

DiGraph with 5 nodes and 8 edges


In [198]:
def execute(
    circuits: qiskit.QuantumCircuit | list[qiskit.QuantumCircuit],
    backend,
    shots: int,
    correct_bitstring: str,
    verbose: bool,
    ) -> List[float]:
    """Executes the input circuit(s) and returns ⟨A⟩, where A = |correct_bitstring⟩⟨correct_bitstring| for each circuit."""

    if not isinstance(circuits, list):
        circuits = [circuits]
    if verbose:
        # Calculate average number of CNOT gates per circuit.
        print(f"Executing {len(circuits)} circuit(s) on {backend}.")
        print(f"Average cnot count in circuits: {get_avg_cnot_count(circuits)}")

    # Store all circuits to run in list to be returned.
    to_run: list[qiskit.QuantumCircuit] = []

    for circuit in circuits:
        circuit_to_run = circuit.copy()
        circuit_to_run.measure_all()
        to_run.append(
            qiskit.transpile(
                circuit_to_run,
                backend=backend,
                initial_layout= get_phys_qubits(circuit.num_qubits),
                optimization_level=0,  # Otherwise RB circuits are simplified to empty circuits.
            )
        )

    if verbose:
        # Calculate average number of CNOT gates per compiled circuit.
        print(f"Average cnot count in compiled circuits: {get_avg_cnot_count(to_run)}")

    # Run and get counts.
    job = backend.run(
        to_run,
        # Reset qubits to ground state after each sample.
        init_qubits=True,
        shots=shots,
    )

    if len(circuits) == 1:
        return [job.result().get_counts().get(correct_bitstring, 0.0) / shots]
    return [
        count.get(correct_bitstring, 0.0) / shots for count in job.result().get_counts()
    ]

In [199]:
def get_cnot_error(edge: tuple[int, int] = None) -> float:
    # cnot_error_prob = 0.01
    rz_error_q0 = noisy_backend.properties().gate_error("rz", qubits=edge[0])
    sqrtx_error_q0 = noisy_backend.properties().gate_error("sx", qubits=edge[0])
    rz_error_q1 = noisy_backend.properties().gate_error("rz", qubits=edge[1])
    ecr_error = noisy_backend.properties().gate_error("ecr", qubits=[edge[0],edge[1]])
    x_error_q1 = noisy_backend.properties().gate_error("x", qubits=edge[1])
    cnot_error_prob = 1 - (1-rz_error_q0)*(1-rz_error_q0)*(1-sqrtx_error_q0)*(1-rz_error_q0)*(1-rz_error_q0)*(1-ecr_error)*(1-rz_error_q1)*(1-x_error_q1)

    print(f"cnot_error_prob for edge {edge}: {cnot_error_prob}")
    return cnot_error_prob


def get_cnot_representation(edge: tuple[int, int]) -> pec.OperationRepresentation:
    cnot_circuit = cirq.Circuit(
        cirq.CNOT(
            cirq.NamedQubit(f"q_{str(edge[0])}"),
            cirq.NamedQubit(f"q_{str(edge[1])}"),
        )
    )

    rep_exact_prob = 1 - np.sqrt(1 - get_cnot_error(edge))
    return pec.represent_operation_with_local_depolarizing_noise(
        cnot_circuit,
        noise_level=rep_exact_prob,
    )


def get_representations(computer: nx.Graph) -> list[pec.OperationRepresentation]:
    return [get_cnot_representation(edge) for edge in computer.edges]

In [200]:
print(computer.edges)

[(0, 1), (1, 0), (1, 2), (2, 1), (2, 3), (3, 2), (3, 4), (4, 3)]


In [201]:
noisy_backend.properties().gate_error("ecr", qubits=[1,2])

0.00878800548814962

In [202]:
print(get_phys_qubits(n_qubits))
print(computer.edges)

[0, 1, 2, 3, 4]
[(0, 1), (1, 0), (1, 2), (2, 1), (2, 3), (3, 2), (3, 4), (4, 3)]


In [203]:
# Hardware backend device type. Supported types are currently "ibmq" but more could be added.
hardware_type: str = "FakeSherbrooke"

backend = {
    "ibmq": "ibm_sherbrooke",
}.get(hardware_type)

In [204]:
local_seed = 10**6 * depth + 10**3 * seed + 3

circuit, correct_bitstring = benchmarks.generate_mirror_circuit(
        nlayers=depth,
        two_qubit_gate_prob=1.0,
        connectivity_graph=computer,
        two_qubit_gate_name="CNOT",
        seed=seed,
        return_type="qiskit",
    )
# Reversed because Qiskit is wrong endian.
correct_bitstring = "".join(map(str, correct_bitstring[::-1]))

(true_value,) = execute(circuit,
    ideal_backend,
    shots,
    correct_bitstring,
    verbose=verbose,
)

(noisy_value,) = execute(circuit,
    noisy_backend,
    shots,
    correct_bitstring,
    verbose=verbose,
)

pec_executor = functools.partial(
        execute,
        backend=noisy_backend,
        shots=shots // num_samples,
        correct_bitstring=correct_bitstring,
        verbose=verbose,
    )
pec_value = pec.execute_with_pec(
        circuit,
        pec_executor,
        representations=get_representations(computer),
        num_samples=num_samples,
        random_state=local_seed,
    )

BackendPropertyError: 'Could not find the desired property for ecr'